## Test of accelerated Monte Carlo ##

In [6]:
import sys
sys.path.insert(0, "../python/")

from particle import particle
from physics import em_physics
from cylinder import cylinder
from analysis import analysis 

import matplotlib.pyplot as plt
import numpy as np
#from numba import jit
import pandas as pd

import json
#import uproot
import csv
import time


### Define the Geometry ###

In [7]:
# define the geometry
radius = 65 # cm
height = 150 #cm
cryostat = cylinder(R=radius,h=height)
# cryostat mass of 5869 kg

radius = 57 # cm
height = 134 #cm
fiducial = cylinder(R=radius,h=height)
#fiducial mass of 4032 kg

cylinder::__init__ Define cylinder with R= 65  and height= 150
cylinder::__init__ Define cylinder with R= 57  and height= 134


### Setup the photon physics ###

In [8]:
# define the physics
em = em_physics()

### Event generation ###

In [3]:
def event_generator(n,energy,emax,mode,nscatter,seed):
    #
    #random seed
    np.random.seed(seed)
    #
    #
    # number of events
    #
    nevent = n
    #
    # gamma energy
    #
    energy = energy # keV
    #
    #
    #number of scaters that are writen to file
    #
    writeout = 4 #scatter events
    #
    #
    
    # event generation loop
    #
    #
    
    tarr=[]
    
    for ieve in range(nevent):

        if ieve%25000 == 0:
            print("generated ",ieve," events")
        #
        # make a particle,
        #
        vrt = mode#'fiducial_scatter'
        
        p = particle(type='gamma',
                     energy=energy, 
                     geometry=cryostat, 
                     fiducial=fiducial, 
                     vrt=vrt, 
                     edep_max=emax,
                     nscatter_max=nscatter,
                     physics=em,
                     debug=False,
                    )

        #
        # propagate the particle and retrieve the intersection points
        #
        p.propagate()

        #
        # store data
        #            
        warr = [ieve,len(p.xint),p.weight,p.edep,p.x0start[0],p.x0start[1],p.x0start[2]]
        
        for i in range(len(p.xint)):    
            warr.extend([p.xint[i][0][0],p.xint[i][0][1],p.xint[i][0][2],p.xint[i][1]])                        
        
        tarr.append(warr[:((writeout*4)+7)])
        
    #
    # close output data file
    #
    df= pd.DataFrame(tarr)
    
    
    if vrt == 'fiducial_scatter':
        
        df.to_csv('../mcdata/testdata_events'+ str(n)+'_mode:vrt'+'_nscatters' +str(nscatter)+'_edep' +str(energy)+ '.' +'csv', index=False, header=False)

    else:
        df.to_csv('../mcdata/testdata_events'+ str(n)+'_mode:nonvrt'+'_nscatters' +str(nscatter)+'_edep' +str(energy)+ '.' +'csv', index=False, header=False)
    


    #print(df)

In [8]:
event_generator(n=1000000,energy=1500,emax=250,mode='fiducial_scatter',nscatter=1,seed=42)
event_generator(n=1000000,energy=1500,emax=250,mode='fiducial_scatter',nscatter=2,seed=43)
event_generator(n=1000000,energy=1500,emax=2500,mode=None,nscatter=2,seed=44)

generated  0  events
generated  25000  events
generated  50000  events
generated  75000  events
generated  100000  events
generated  125000  events
generated  150000  events
generated  175000  events
generated  200000  events
generated  225000  events
generated  250000  events
generated  275000  events
generated  300000  events
generated  325000  events
generated  350000  events
generated  375000  events
generated  400000  events
generated  425000  events
generated  450000  events
generated  475000  events
generated  500000  events
generated  525000  events
generated  550000  events
generated  575000  events
generated  600000  events
generated  625000  events
generated  650000  events
generated  675000  events
generated  700000  events
generated  725000  events
generated  750000  events
generated  775000  events
generated  800000  events
generated  825000  events
generated  850000  events
generated  875000  events
generated  900000  events
generated  925000  events
generated  950000  e

## Make reference simulation##

This is basically the same as the normal simulation; only this saves the values every x steps to prevent memory loss. (Value is a tradeoff between memory and computation time.)

In [4]:
def ref_event_generator(n=10000000,energy=10000,emax=250,mode='fiducial_scatter',nscatter=1,seed=42):

    #%%prun -D my.profile
    #
    #random seed
    np.random.seed(seed)
    #
    # number of events
    #
    nevent = n
    #
    # gamma energy
    #
    energy = energy # keV
    #
    #
    #number of scaters that are writen to file
    #
    writeout = 4 #scatter events
    #
    #
    
    # event generation loop
    #
    #
    
    tarr=[]
    file = '../mcdata/reference'+str(n)+'.' +'csv'
    open(file, 'w').close()
    
    for ieve in range(nevent):

        if ieve%25000 == 0:
            print("generated ",ieve," events")
        #
        # make a particle,
        #
        seed = None
        vrt = mode #'fiducial_scatter'
        if vrt==None: nscatter = 20
        
        p = particle(type='gamma',
                     energy=energy, 
                     geometry=cryostat, 
                     fiducial=fiducial, 
                     vrt=vrt, 
                     edep_max=emax,
                     nscatter_max=nscatter,
                     physics=em,
                     debug=False,
                     seed=seed #ieve*42 random seed for testing None for not giving a seed
                    )

        #
        # propagate the particle and retrieve the intersection points
        #
        p.propagate()

        #
        # store data
        #            
        warr = [ieve,len(p.xint),p.weight,p.edep,p.x0start[0],p.x0start[1],p.x0start[2]]
        
        
        for i in range(len(p.xint)):    
            warr.extend([p.xint[i][0][0],p.xint[i][0][1],p.xint[i][0][2],p.xint[i][1]])
            
        
        
        tarr.append(warr[:((writeout*4)+7)])
        
        # Write every x event to file
        if ieve%25000 == 0:
            df= pd.DataFrame(tarr)
            df.to_csv(file,mode='a',index=False, header=False)
            tarr=[]
            del(df)

    #print(df)
    return

In [9]:
ref_event_generator(n=100000)

generated  0  events
generated  25000  events
generated  50000  events
generated  75000  events
generated  100000  events
generated  125000  events
generated  150000  events
generated  175000  events
generated  200000  events
generated  225000  events
generated  250000  events
generated  275000  events
generated  300000  events
generated  325000  events
generated  350000  events
generated  375000  events
generated  400000  events
generated  425000  events
generated  450000  events
generated  475000  events
generated  500000  events
generated  525000  events
generated  550000  events
generated  575000  events
generated  600000  events
generated  625000  events
generated  650000  events
generated  675000  events
generated  700000  events
generated  725000  events
generated  750000  events
generated  775000  events
generated  800000  events
generated  825000  events
generated  850000  events
generated  875000  events
generated  900000  events
generated  925000  events
generated  950000  e

generated  7650000  events
generated  7675000  events
generated  7700000  events
generated  7725000  events
generated  7750000  events
generated  7775000  events
generated  7800000  events
generated  7825000  events
generated  7850000  events
generated  7875000  events
generated  7900000  events
generated  7925000  events
generated  7950000  events
generated  7975000  events
generated  8000000  events
generated  8025000  events
generated  8050000  events
generated  8075000  events
generated  8100000  events
generated  8125000  events
generated  8150000  events
generated  8175000  events
generated  8200000  events
generated  8225000  events
generated  8250000  events
generated  8275000  events
generated  8300000  events
generated  8325000  events
generated  8350000  events
generated  8375000  events
generated  8400000  events
generated  8425000  events
generated  8450000  events
generated  8475000  events
generated  8500000  events
generated  8525000  events
generated  8550000  events
g

generated  5275000  events
generated  5300000  events
generated  5325000  events
generated  5350000  events
generated  5375000  events
generated  5400000  events
generated  5425000  events
generated  5450000  events
generated  5475000  events
generated  5500000  events
generated  5525000  events
generated  5550000  events
generated  5575000  events
generated  5600000  events
generated  5625000  events
generated  5650000  events
generated  5675000  events
generated  5700000  events
generated  5725000  events
generated  5750000  events
generated  5775000  events
generated  5800000  events
generated  5825000  events
generated  5850000  events
generated  5875000  events
generated  5900000  events
generated  5925000  events
generated  5950000  events
generated  5975000  events
generated  6000000  events
generated  6025000  events
generated  6050000  events
generated  6075000  events
generated  6100000  events
generated  6125000  events
generated  6150000  events
generated  6175000  events
g

generated  0  events
generated  25000  events
generated  50000  events


KeyboardInterrupt: 